##데이터 불러오기

### 데이터 전처리 후

In [40]:
import pandas as pd
import numpy as np
import re
import json

In [41]:
path = "/content/drive/MyDrive/pre_onboarding/Text_summarization/sports_news_data.csv"
data = pd.read_csv(path)
data = data[["TITLE","CONTENT"]]
data= data.dropna(axis=0)
data = data.drop_duplicates(["CONTENT"], keep='first').reset_index(drop=True)

data.loc[209]

TITLE              [오피셜] 서울 이랜드 FC, 울산 김인성 영입... “동기부여 필요했다”
CONTENT    [스포탈코리아] 이적시장 마지막 날, 서울 이랜드 FC는 오랜 기다림 속에 스피드 ...
Name: 209, dtype: object

In [42]:
# 000 기자, 통신원 부분 삭제 
data_li = []

for text in  data['CONTENT']:
  if '기자 =' in text:
     text = text.split("기자 =")[1]
     data_li.append(text)

  elif '기자=' in text:  
     text = text.split("기자=")[1]
     data_li.append(text)

  elif '통신원=' in text:  
     text = text.split("통신원=")[1]
     data_li.append(text)
  else: 
     data_li.append(text)

data['CONTENT']=data_li  

In [43]:
# 전처리 함수
import re

def preprocess_sentence(sentence):
    sentence = re.sub("(<([^>]+)>)", '', sentence) # <p>같은 <> 제거
    sentence = re.sub("\[.*\]", '', sentence) # []안에 있는것 제거
    sentence = re.sub("\n|\t|\r", '', sentence) # 공백 제거
    sentence = re.sub("\↓", '하락', sentence) # 기호 한글로 대체
    sentence = re.sub("\↑", '상승', sentence) 
    sentence = re.sub("\→", '에서', sentence) 
    sentence = re.sub("美", '미국', sentence) # 한자 한글로 대체
    sentence = re.sub("韓", '한국', sentence) 
    sentence = re.sub("日", '일본', sentence) 
    sentence = re.sub("中", '중국', sentence) 
    sentence = re.sub("英", '영국', sentence) 
    sentence = re.sub("주요 뉴스", '', sentence) # 주요뉴스 단어 제거
    sentence = re.sub('(\d{2}).(\d{2}).(\d{2})', r'\1-\2-\3', sentence) # 11.02.02 -> 11-02-02 로 형태 변경
    sentence = re.sub('(\d)[.](\d)', r'\1온점\2',sentence) # 8.5 -> 8온점5 형태로 변경. 후에 split후 다시 변경예정
    return sentence

In [44]:
data['CONTENT']= data['CONTENT'].apply(preprocess_sentence)
data['CONTENT'] = data["CONTENT"].str.split("사진=").str[0] # 사진 제거
data['TITLE']= data['TITLE'].apply(preprocess_sentence)

data = data.replace('[^A-Za-z0-9가-힣.()"-:]+', ' ', regex=True) # 한글,영어, 숫자, 일부 특수문제 제외 제거

In [161]:
# 문장으로 split 후 리스트로 변경 
content_split = data['CONTENT'].str.split(".").values.tolist()

# split 하느라 생긴 맨뒤 nan 제거 
content_split = [s[:-1] for s in content_split]

content = []
zero_index = []
index = []
drop_li= []
full_content = []
lead = []

for idx,line in enumerate(content_split):
    text = []

    for s in line:
        s = re.sub('(\d)온점(\d)', r'\1.\2', s)
        text.append(s.strip())
    full_content.append(text)
    lead.append(text[:3])
    if len(text) > 7:
        content.append(text)
        index.append(idx)
    elif len(line) == 0 :
        zero_index.append(idx)
    else:
         drop_li.append(idx)

#  len(text) < 7 인 문장 제거
# data= data.drop(drop_li, axis = 0)

# content 문장단위 list로 저장
# data['CONTENTS'] = content

In [162]:
data['FULL_CONTENTS'] = full_content
data['LABEL'] = lead

In [163]:
data['LABEL'][451] = ["소대현(울산 사무국장) 씨 빙모상"]
data['FULL_CONTENTS'][451] = ["소대현(울산 사무국장) 씨 빙모상"]
data['LABEL'][920] = ["이영호(연합뉴스 스포츠부 차장)씨 부친상"]
data['FULL_CONTENTS'][920] = ["이영호(연합뉴스 스포츠부 차장)씨 부친상"]
data['LABEL'][4522] = ["한국프로축구연맹(총재 권오갑)이 전주시설관리공단(이사장 구대식)에 감사패를 전달했다."]
data['FULL_CONTENTS'][4522] = ["한국프로축구연맹(총재 권오갑)이 전주시설관리공단(이사장 구대식)에 감사패를 전달했다."]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [102]:
#길이 확인
data['len_title'] =  data['TITLE'].apply(lambda x: len(x))
data['len_content'] =  data['CONTENT'].apply(lambda x: len(x))
data['len_full'] =  data['FULL_CONTENTS'].apply(lambda x: len(x))

In [103]:
#평균 길이
print(data['len_title'].mean())
print(data['len_content'].mean())
print(data['len_full'].mean())

34.43314917127072
866.1755801104972
19.19790055248619


In [104]:
# 최대 길이
print('title :',data['len_title'].max(),',',data['len_title'].min())
print('content :',data['len_content'].max(),',', data['len_content'].min())
print('full_contents :',data['len_full'].max(),',',data['len_full'].min())

title : 57 , 14
content : 15522 , 5
full_contents : 251 , 1


In [105]:
#본문 문장 길이가 0인 경우 대체
data[data['len_full']==0]

,TITLE,CONTENT,FULL_CONTENTS,LABEL,len_title,len_content,len_full,title_count,content_count,label_count


In [210]:
#어절 개수
data['title_count'] = data['TITLE'].str.split(' ')
data['content_count'] = data['CONTENT'].str.split(' ')
data['label_count'] = data['LABEL'].str.split(' ')
data['title_count']  =  data['title_count'].apply(lambda x: len(x))
data['content_count'] =  data['content_count'].apply(lambda x: len(x))
data['label_count'] = data['label_count'].apply(lambda x: len(x))

In [211]:
#평균 어절 개수
print(data['title_count'].mean())
print(data['content_count'].mean())
print(data['label_count'].mean())

9.400773480662984
197.9636464088398
32.36917127071823


In [ ]:
!pip install konlpy 

In [172]:
from konlpy.tag import Komoran
komoran = Komoran()

In [182]:
#품사 처리
def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    #words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

def komoran_noun(sent):
    words = komoran.nouns(sent)
    return words

In [201]:
data['LABEL'] = data['LABEL'].apply(lambda x: ",".join(x))

In [203]:
data['token_title'] = data['TITLE'].apply(komoran_tokenizer)
data['token_content'] = data['CONTENT'].apply(komoran_tokenizer)
data['token_label'] = data['LABEL'].apply(komoran_tokenizer)

In [204]:
data['noun_title'] = data['TITLE'].apply(komoran_noun)
data['noun_content'] = data['CONTENT'].apply(komoran_noun)
data['noun_label'] = data['LABEL'].apply(komoran_noun)

In [205]:
data['token_title'] = data['token_title'].apply(lambda x: len(x))
data['token_content'] = data['token_content'].apply(lambda x: len(x))
data['token_label'] = data['token_label'].apply(lambda x: len(x))
data['noun_title'] = data['noun_title'].apply(lambda x: len(x))
data['noun_content'] = data['noun_content'].apply(lambda x: len(x))
data['noun_label'] = data['noun_label'].apply(lambda x: len(x))

In [206]:
data['title_noun_ratio']  =  data['noun_title']/data['token_title'] 
data['content_noun_ratio'] =  data['noun_content']/data['token_content']
data['label_noun_ratio'] =  data['noun_label']/data['token_label']

In [207]:
#전체 타이틀, 본문 평균 명사 사용률
print(data['title_noun_ratio'].mean())
print(data['content_noun_ratio'].mean())
print(data['label_noun_ratio'].mean())

0.48577895089458323
0.34339328612234205
0.3943054029724862


In [212]:
data

,TITLE,CONTENT,FULL_CONTENTS,LABEL,len_title,len_content,len_full,title_count,content_count,label_count,token_title,token_content,noun_title,noun_content,title_noun_ratio,content_noun_ratio,token_label,noun_label,label_noun_ratio
0,스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마...,[유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마...,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,29,809,18,9,188,30,10,409,7,150,0.700000,0.366748,68,31,0.455882
1,"디 마리아 없다 유로X코파 베스트11, 이탈리아만 7명",유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.지난...,"[유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까, 지...","유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까,지난달...",31,714,17,8,150,28,15,337,9,131,0.600000,0.388724,60,23,0.383333
2,슈퍼컴퓨터 예측 맨시티 우승-맨유 4위 토트넘은 6위,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.영국 매체 스포츠 바...,"[새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다, 영국 매체 스포츠 ...","새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다,영국 매체 스포츠 바이...",31,646,16,9,138,23,14,338,8,110,0.571429,0.325444,60,25,0.416667
3,"이재성, 완벽한 프로 마인츠서 성공할 것 킬 디렉터의 애정 듬뿍 응원",홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.이재성은 2...,"[홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다, 이재성은 ...","홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다,이재성은 20...",39,834,26,12,191,21,19,444,9,130,0.473684,0.292793,47,17,0.361702
4,"홈킷과 딴판 바르사 팬들, NEW 어웨이 셔츠 호평 가장 좋아하는 색",FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽다는 반응이다....,"[FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다, 팬들은 만족스럽다는 반응이다,...","FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다,팬들은 만족스럽다는 반응이다,바르...",40,532,16,13,124,19,16,279,7,84,0.437500,0.301075,49,19,0.387755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9045,"이제 홈팬 야유 받는 먹튀 선수, 차비 감독 조차 그만 해라",FC바르셀로나(이하 바르사) 팬들에게 우스망 뎀벨레는 밉상이 되어버렸다. 바르사는...,"[FC바르셀로나(이하 바르사) 팬들에게 우스망 뎀벨레는 밉상이 되어버렸다, 바르사는...","FC바르셀로나(이하 바르사) 팬들에게 우스망 뎀벨레는 밉상이 되어버렸다,바르사는 7...",34,475,13,12,107,29,16,265,8,78,0.500000,0.294340,79,28,0.354430
9046,"성남, 만 17세 유스 김지수와 준프로 계약",성남FC가 만17세 2004년생 수비수 김지수와 준프로 계약을 체결했다. 김지수는 ...,"[성남FC가 만17세 2004년생 수비수 김지수와 준프로 계약을 체결했다, 김지수는...","성남FC가 만17세 2004년생 수비수 김지수와 준프로 계약을 체결했다,김지수는 1...",26,811,14,9,188,32,12,404,6,157,0.500000,0.388614,82,29,0.353659
9047,"오베르마스, 아약스서 쫓겨난다...'여성 동료들에게 부적절한 메시지'",'레전드' 마르크 오베르마스가 아약스에서 쫓겨났다. 이유는 굉장히 굴욕적이었다.아...,"['레전드' 마르크 오베르마스가 아약스에서 쫓겨났다, 이유는 굉장히 굴욕적이었다, ...","'레전드' 마르크 오베르마스가 아약스에서 쫓겨났다,이유는 굉장히 굴욕적이었다,아약스...",38,737,21,6,167,17,19,391,5,114,0.263158,0.291560,49,17,0.346939
9048,"바르사 차비 감독, 트라오레 데뷔전 활약에 깜짝...""몸이 야수 같아""",FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른 아다마 트라오레를 극찬했다....,"[FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른 아다마 트라오레를 극찬했다,...","FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른 아다마 트라오레를 극찬했다,바...",39,825,18,9,180,38,18,426,7,142,0.388889,0.333333,87,34,0.390805


##데이터 확인하기

###최대 길이
- 타이틀 : 62
- 본문 : 15995
- 요약문 : 1043

In [10]:
# 가장 긴 데이터
data['TITLE']

'힘든 원정길에도 \'귀중한 승리\' 정우영, "우리가 할 수 있는 걸 하고자 했다"'

In [21]:
def longestWord(some_list):
    count = 0    #You set the count to 0
    for i in some_list: # Go through the whole list
        if len(i) > count: #Checking for the longest word(string)
            count = len(i)
            word = i
    return (f"the longest string is {word}, length is {count}".format(word,count))

In [22]:
longestWord(data['TITLE'])

"the longest string is  K리그1 37R '주민규 K리그 100호골' 제주, 수원FC에 1-0 승...'4위 확보+ACL 진출 청신호', length is 62"

In [ ]:
longestWord(data['CONTENT'])

In [24]:
longestWord(data['LABEL'])

"the longest string is ['벤제마 시즌별 드리블 돌파 시도 / 성공률(90분당 평균 라 리가 기록 기준)2017/18 - 1.8회 / 73.3%(호날두의 마지막 시즌)2018/19 - 2.5회 / 62.4%2019/20 - 2.0회 / 54.2%2020/21 - 1.8회 / 65.0%2021/22 1.5회 / 76.0%**19라운드 종료 후올 시즌 라 리가 최전방 공격수 드리블 성공 후 슈팅 횟수 TOP 5(19라운드 종료 후)8회 - 카림 벤제마(레알 마드리드)6회 - 알렉산데르 이삭(레알 소시에다드)6회 - 루카스 보예(엘체)5회 - 멤피스 데파이(바르셀로나)5회 - 라울 데 토마스(에스파뇰)그러다 보니 벤제마는 자연스럽게 득점 상황에 관여하는 빈도도 라 리가 정상급 수준으로 올라섰다', '벤제마 시즌별 총 압박 시도 / 파이널 서드 안 압박 시도(90분당 평균 라 리가 기록 기준)2017/18 - 16.7회 / 8.4회(호날두의 마지막 시즌)2018/19 - 13.5회 / 7.2회2019/20 - 9.1회 / 4.6회2020/21 - 7.6회 / 3.9회2021/22 - 9.6회 / 5.4회**19라운드 종료 후레알이 최전방 공격수 벤제마의 압박이 눈에 띌 정도로 느슨해진 상태로도 올 시즌 전반기를 마친 현시점부터 일찌감치 라 리가 우승이 유력해졌을 정도로 안정적인 전력을 자랑하는 이유는 팀 전체가 상대의 후방 빌드업을 방해하는 압박 방식 자체를 바꿨기 때문이다', '벤제마 시즌별 패스 기록(90분당 평균 키패스 / 페널티 지역 진입 패스 / 침투성 패스)2017/18 - 1.8회 / 1.5회 / 3.2회(호날두의 마지막 시즌)2018/19 - 1.4회 / 1.7회 / 2.9회2019/20 - 1.7회 / 1.6회 / 3.1회2020/21 - 1.1회 / 1.1회 / 2.5회2021/22 - 2.1회 / 3.1회 / 4.7회*침투성 패스 상대 골대를 향해 움직이는 최소 약 10m 길이의 전진 패스*19라운드 종료 후지난 4 5년을 거치며

###품사 확인

In [17]:
!pip install konlpy 

In [18]:
from konlpy.tag import Komoran
komoran = Komoran()

In [19]:
def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    #words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

In [26]:
komoran_tokenizer(data['TITLE'][1])

['‘/SS',
 '디/NNP',
 '마리아/NNP',
 '없/VA',
 '다/EC',
 '’/SS',
 '유로/NNP',
 'X/SL',
 '코/NNG',
 '파/NNG',
 '베스트/NNP',
 '11/NNP',
 ',/SP',
 '이탈리아/NNP',
 '만/JX',
 '7/SN',
 '명/NNB']